In [39]:
import polars as pl
import numpy as np
import plotly.express as px
from datetime import datetime, timedelta
import plotly.graph_objects as go
import polars as pl


df_crsp = pl.scan_parquet('./Datasets/crsp')
df_cstat = pl.scan_parquet('./Datasets/compustat/fundamentals_quarterly_all')
df_omni_key = pl.scan_parquet('./Datasets/meta')

print(df_crsp.collect_schema().names())
print(df_cstat.collect_schema().names())

['PERMNO', 'date', 'NAMEENDT', 'SHRCD', 'EXCHCD', 'NCUSIP', 'TICKER', 'COMNAM', 'SHRCLS', 'PRIMEXCH', 'TRDSTAT', 'SECSTAT', 'PERMCO', 'CUSIP', 'DCLRDT', 'DLAMT', 'DLPDT', 'DLSTCD', 'PAYDT', 'RCRDDT', 'SHRFLG', 'DISTCD', 'DIVAMT', 'FACPR', 'FACSHR', 'ACPERM', 'ACCOMP', 'SHRENDDT', 'NWPERM', 'DLRETX', 'DLPRC', 'DLRET', 'BIDLO', 'ASKHI', 'PRC', 'VOL', 'RET', 'ASK', 'SHROUT', 'CFACPR', 'CFACSHR', 'OPENPRC', 'NUMTRD', 'RETX', 'vwretd', 'vwretx', 'ewretd', 'ewretx', 'sprtrn']
['costat', 'curcdq', 'datafmt', 'indfmt', 'consol', 'gvkey', 'datadate', 'conm', 'tic', 'cusip', 'cik', 'exchg', 'fyr', 'fic', 'add1', 'add2', 'add3', 'add4', 'addzip', 'busdesc', 'city', 'conml', 'county', 'dldte', 'dlrsn', 'ein', 'fax', 'fyrc', 'ggroup', 'gind', 'gsector', 'gsubind', 'idbflag', 'incorp', 'ipodate', 'loc', 'naics', 'phone', 'prican', 'prirow', 'priusa', 'sic', 'spcindcd', 'spcseccd', 'spcsrc', 'state', 'stko', 'weburl', 'acctchgq', 'acctstdq', 'adrrq', 'ajexq', 'ajpq', 'apdedateq', 'bsprq', 'compstq', 

In [41]:
# compustat rolling sum
cstat_clean = df_cstat.select(['gvkey', 'datadate', 'gsector', 'tic', 'cusip', 'conm', 'cik', 'fyearq', 'fqtr', 
                 'rdq', 'indfmt', 'datafmt', 'consol', 'curcdq', 'costat', 'prccq', 'cshoq',
                 'ajexq', 'mkvaltq', 'atq', 'actq', 'cheq', 'rectq', 'invtq', 'ppentq',
                 'ltq', 'lctq', 'dlcq', 'dlttq', 'apq', 'txdbq','seqq', 'ceqq', 'pstkq',
                 'saleq', 'cogsq', 'xsgaq', 'xrdq', 'dpq', 'xintq','oiadpq', 'piq', 'txtq', 
                 'niq', 'ibq','epsfxq', 'oancfy', 'capxy', 'dvy', 'aqcy', 'epspxq', 'opepsq'])
cstat_clean.filter(pl.col('gsector').is_not_null()).tail().collect()

gvkey,datadate,gsector,tic,cusip,conm,cik,fyearq,fqtr,rdq,indfmt,datafmt,consol,curcdq,costat,prccq,cshoq,ajexq,mkvaltq,atq,actq,cheq,rectq,invtq,ppentq,ltq,lctq,dlcq,dlttq,apq,txdbq,seqq,ceqq,pstkq,saleq,cogsq,xsgaq,xrdq,dpq,xintq,oiadpq,piq,txtq,niq,ibq,epsfxq,oancfy,capxy,dvy,aqcy,epspxq,opepsq
str,date,str,str,str,str,str,i32,i16,date,str,str,str,str,str,"decimal[24,12]","decimal[18,4]","decimal[24,12]","decimal[18,4]","decimal[18,4]","decimal[18,4]","decimal[18,4]","decimal[18,4]","decimal[18,4]","decimal[18,4]","decimal[18,4]","decimal[18,4]","decimal[18,4]","decimal[18,4]","decimal[18,4]","decimal[18,4]","decimal[18,4]","decimal[18,4]","decimal[18,4]","decimal[18,4]","decimal[18,4]","decimal[18,4]","decimal[18,4]","decimal[18,4]","decimal[18,4]","decimal[18,4]","decimal[18,4]","decimal[18,4]","decimal[18,4]","decimal[18,4]","decimal[18,4]","decimal[18,4]","decimal[18,4]","decimal[18,4]","decimal[18,4]","decimal[18,4]","decimal[18,4]"
"""266160""",2025-12-31,"""40""","""CZWI""","""174903104""","""CITIZENS COMMUNITY BANCORP""","""0001367859""",2025,4,2026-01-26,"""INDL""","""STD""","""C""","""USD""","""A""",17.820000000000,9.6170,1.000000000000,171.3749,1781.7550,null,118.8530,1317.9240,5.8110,16.3570,1593.8160,null,null,null,1524.0990,0.0000,187.9390,187.9390,0.0000,24.4630,8.9060,7.1550,null,null,null,8.4020,4.8850,0.6140,4.2710,4.2710,0.4400,null,null,null,null,0.4400,0.4400
"""266214""",2025-12-31,"""40""","""CNS""","""19247A100""","""COHEN & STEERS INC""","""0001284812""",2025,4,2026-01-22,"""INDL""","""STD""","""C""","""USD""","""A""",62.780000000000,51.0040,1.000000000000,3202.0311,null,null,null,null,null,null,null,null,null,null,null,null,562.0000,562.0000,0.0000,143.8030,100.9580,null,null,2.5350,0.0000,40.3100,42.2880,11.5850,34.8790,34.8790,0.6800,null,null,null,null,0.6800,0.6800
"""294524""",2025-12-31,"""15""","""LYB""","""N53745100""","""LYONDELLBASELL INDUSTRIES NV""","""0001489393""",2025,4,2026-01-30,"""INDL""","""STD""","""C""","""USD""","""A""",43.300000000000,322.0000,1.000000000000,13942.6000,34003.0000,10868.0000,3449.0000,2517.0000,3533.0000,17347.0000,23796.0000,6129.0000,814.0000,13451.0000,2694.0000,2316.0000,10082.0000,10082.0000,0.0000,7091.0000,6335.0000,397.0000,33.0000,385.0000,132.0000,-26.0000,-142.0000,-8.0000,-142.0000,-136.0000,-0.4300,2262.0000,1878.0000,1764.0000,null,-0.4300,-0.2600
"""317264""",2025-12-31,"""10""","""LPG""","""Y2106R110""","""DORIAN LPG LTD""","""0001596993""",2025,3,2026-02-05,"""INDL""","""STD""","""C""","""USD""","""A""",24.340000000000,42.7440,1.000000000000,1040.3890,1777.7390,392.2860,294.4920,73.3500,2.2510,1328.0300,692.1100,162.4620,145.5590,528.0680,6.6910,0.0000,1085.6290,1085.6290,0.0000,120.6500,42.3170,10.7800,null,16.2420,6.8300,51.3110,47.1890,0.0000,47.1890,47.1890,1.1100,127.9960,29.6650,75.2490,0.0000,1.1100,1.1100
"""326688""",2025-12-31,"""20""","""NVT""","""G6700G107""","""NVENT ELECTRIC PLC""","""0001720635""",2025,4,2026-02-06,"""INDL""","""STD""","""C""","""USD""","""A""",101.970000000000,161.3600,1.000000000000,16453.8792,6851.9000,1639.6000,237.5000,693.0000,null,434.5000,3121.7000,1003.5000,13.8000,1546.0000,358.9000,232.0000,3730.2000,3730.2000,0.0000,1066.7000,677.8000,219.7000,21.3000,null,19.1000,169.2000,156.3000,40.5000,118.8000,115.8000,0.7100,465.2000,93.3000,130.4000,975.7000,0.7200,0.7400


In [42]:
# earnings TTM (trailing twelve month) rolling sum: cont.
result = cstat_clean.select(['tic', 'gvkey','mkvaltq', 'datadate', 'epspxq']) \
    .with_columns(
        pl.col("epspxq")
        .cast(pl.Float64)
        .rolling_sum(window_size=4)
        .over('gvkey')
        .alias('TTM')
    ).filter(
        pl.col("TTM")
        .is_not_null(),
        pl.col('mkvaltq')
        .cast(pl.Float64) > 50000
    )

sampled_gvkeys = result.select(pl.col('gvkey').unique().sample(10))

result = result.join(sampled_gvkeys, on='gvkey', how="semi") \
    .sort(pl.col('TTM')) \
    .collect()

fig = go.Figure(data=[go.Table(
    header=dict(
        values=result.columns,
        align='left',
    ),
    cells=dict(
        values=[result[col] for col in result.columns], # Pass data as a list of columns
        align='left',
        height=30
    )
)])
fig.update_layout(title_text="Sampled Rolling Sum`")
fig.show()

In [69]:
# finding momentum_score
filtered_crsp = df_crsp.select(['PERMNO', 'TICKER', 'date', 'EXCHCD', 'BIDLO', 'ASKHI', 'VOL', 'SHROUT', 'OPENPRC', 'PRC'])

pl.set_random_seed(1)
sampled_permnos = filtered_crsp.select(pl.col('PERMNO').unique().sample(1)).collect()

filtered_crsp = (
    filtered_crsp
    .join(sampled_permnos.lazy(), on='PERMNO', how='semi')
    .with_columns(
        pl.col('date').str.to_date()
    )
    .sort("date")
    .with_columns(
        pl.col('PRC')
        .rolling_sum_by('date', window_size='3d', closed='right')
        .over('PERMNO')
        .alias('3D-rolling-sum')
    )
    .drop_nulls()
    .collect()
)

fig = go.Figure(data=[go.Table(
    header=dict(
        values=filtered_crsp.columns,
        align='left'
    ),
    cells=dict(
        values=[filtered_crsp[col] for col in filtered_crsp.columns],
        align='left',
        height=30
    )
)])

fig.update_layout(title_text='CRSP 1-Random-Stock Sampled Data')
fig.show()